In [1]:
from scripts import dataset, tools, iterator
from models import alexnet, regressionnet
from copy import copy
from tqdm import tqdm
import math, os
import tensorflow as tf
import numpy as np
tools.etc.set_GPU("3")

In [2]:
with tf.device("/gpu:3"):
    net, loss_op, pose_loss_op, train_op = regressionnet.create_regression_net(
        batch_size=100,
        data_shape=(227,227,3),
        num_joints=14)

    with net.graph.as_default():
        saver = tf.train.Saver()
        train_it = iterator.create("./dataset/train_MET_L.csv", batch_size=100, mode="Regression", Shuffle=True)
        test_it = iterator.create("./dataset/test_MET_L.csv", batch_size=100, mode="Regression", Shuffle=True)
        #train_it = dataset.iterator("./dataset/train_MET_L.csv", batch_size=100, mode="Regression", Shuffle=True)
        #test_it = dataset.iterator("./dataset/test_MET_L.csv", batch_size=100, mode="Regression", Shuffle=True)

        summary_writer = tf.summary.FileWriter("./out/", net.sess.graph)
        summary_op = tf.summary.merge_all()
        #fc_train_op = net.graph.get_operation_by_name('fc_train_op')

In [3]:
fix_conv_iter = 10000
summary_step = 50

with net.graph.as_default():
    summary_writer = tf.summary.FileWriter("./out", net.sess.graph)
    summary_op = tf.summary.merge_all()

global_step = None

#net.sess.run(train_it.iterator.make_initializer(train_it.data))
#net.sess.run(test_it.iterator.make_initializer(test_it.data))
cur_train_op = train_op

In [ ]:
with tf.device("/gpu:3"):
    with tqdm(total = fix_conv_iter) as pbar:
        for step in range(fix_conv_iter):
            net.sess.run(train_it.iterator.make_initializer(train_it.data))
            cost = 0.
            cnt = 0
            for _ in range(len(train_it.img_path)//train_it.batch_size):
                batch = net.sess.run(train_it.iterator.get_next())

                global_step, summary_str, _, loss_value = net.sess.run(
                    [net.global_iter_counter,summary_op,cur_train_op,pose_loss_op],
                    feed_dict={net.x : batch[0],
                               'PoseInput/joints_ground_truth:0' : batch[1],
                               'PoseInput/joints_is_valid:0': batch[2],
                               'lr/conv_lr:0': 0.,
                               'lr/fc_lr:0': 0.
                                              })
                cost+=loss_value
                cnt+=1
            pbar.update(1)
            pbar.set_description("[ Step : "+str(step+1)+"]")
            pbar.set_postfix_str(cost/cnt)

[ Step : 539]:   5%|▌         | 539/10000 [1:17:53<24:12:49,  9.21s/it, 3702.70361328]